Notebook to do RHF-DMRG of H2 for test purposes

In [1]:
from pyscf import gto, scf, mcscf
import numpy as np
import matplotlib.pyplot as plt
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import os 

In [3]:
# to delete the tmp files

folder_path = "./tmp"

for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a directory) and delete it
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")

Deleted: ./tmp/F.MPS.GS.0
Deleted: ./tmp/F.MPS.GS.1
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.0
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.0
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.1
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.1
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.2
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.2
Deleted: ./tmp/F.MPS.GS.-1
Deleted: ./tmp/GS-mps_info.bin
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.LEFT.0
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.RIGHT.0
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.LEFT.1
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.RIGHT.1
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.LEFT.2
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.RIGHT.2
Deleted: ./tmp/F.MPS.PDM-KET@TMP.0
Deleted: ./tmp/F.MPS.PDM-KET@TMP.1
Deleted: ./tmp/F.MPS.PDM-KET@TMP.-1
Deleted: ./tmp/F0.PART.NPDM.NPDM.FRAG.0.fpc


In [2]:
#RHF-DMRG
H2 = gto.M(atom=[["H", 0, 0, 0],
                  ["H", 1.57, 0, 0]], basis = 'sto-3g', spin = 0)
rhf=scf.RHF(H2).run()

norbs, nelecas = (2, 2)
ref_ucas = mcscf.CASSCF(rhf, norbs, nelecas).mc1step()[0]


bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(rhf,
    ncore=0, ncas=None, g2e_symm=1)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket = driver.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy = driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy = %20.15f' % energy)

converged SCF energy = -0.890301503470727
CASSCF energy = -0.987594783712069
CASCI E = -0.987594783712069  E(CI) = -1.32465033206876  S^2 = 0.0000000
integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =         22

Build MPO | Nsites =     2 | Nterms =         22 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     2 .. Mmpo =    12 DW = 0.00e+00 NNZ =       12 SPT = 0.0000 Tmvc = 0.000 T = 0.004
 Site =     1 /     2 .. Mmpo =     1 DW = 0.00e+00 NNZ =       12 SPT = 0.0000 Tmvc = 0.000 T = 0.003
Ttotal =      0.007 Tmvc-total = 0.000 MPO bond dimension =    12 MaxDW = 0.00e+00
NNZ =           24 SIZE =           24 SPT = 0.0000

Rank =     0 Ttotal =      0.024 MPO method = FastBipartite bond dimension =      12 NNZ =           24 SIZE =           24 SPT = 0.0000

Sweep =    0 | Direction =  forward | Bond dimension =   10 | Noise =  1.00e-04 | Dav threshold =  1.00e-08
Time elapsed =      0.036 | E =      -0.9875947837 | DW = 0.00000e+00

Sweep =    1 |

In [3]:
#do rhf - dmrg in Sz mode
H2 = gto.M(atom=[["H", 0, 0, 0],
                  ["H", 1.57, 0, 0]], basis = 'sto-3g', spin = 0)
rhf=scf.RHF(H2).run()

norbs, nelecas = (2, 2)
ref_ucas = mcscf.CASSCF(rhf, norbs, nelecas).mc1step()[0]


bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(rhf,
    ncore=0, ncas=None, g2e_symm=1)

driver_sz = DMRGDriver(scratch="./tmp2", symm_type=SymmetryTypes.SZ, n_threads=4)
driver_sz.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo_sz = driver_sz.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket_sz = driver_sz.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy_sz = driver_sz.dmrg(mpo_sz, ket_sz, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy SZ mode = %20.15f' % energy_sz)



converged SCF energy = -0.890301503470727
CASSCF energy = -0.987594783712069
CASCI E = -0.987594783712069  E(CI) = -1.32465033206876  S^2 = 0.0000000
integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =         54

Build MPO | Nsites =     2 | Nterms =         54 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     2 .. Mmpo =    22 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.003
 Site =     1 /     2 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.004 Tmvc-total = 0.000 MPO bond dimension =    22 MaxDW = 0.00e+00
NNZ =           44 SIZE =           44 SPT = 0.0000

Rank =     0 Ttotal =      0.013 MPO method = FastBipartite bond dimension =      22 NNZ =           44 SIZE =           44 SPT = 0.0000

Sweep =    0 | Direction =  forward | Bond dimension =   10 | Noise =  1.00e-04 | Dav threshold =  1.00e-08
Time elapsed =      0.007 | E =      -0.9875947837 | DW = 0.00000e+00

Sweep =    1 |

In [3]:
#Mo_coeff=rhf.mo_coeff
print(h1e)

#np.save('/workspaces/MRA-OrbitalOptimization/h2_test/coeff/h2_test_coeff_1.npy', Mo_coeff)
#np.save('/workspaces/MRA-OrbitalOptimization/h2_test/coeff/h2_test_coeff_2.npy', Mo_coeff)


[[-8.86175276e-01 -2.22044605e-16]
 [-1.11022302e-16 -6.68594018e-01]]


In [3]:
alpha_coeff = np.load('/workspaces/MRA-OrbitalOptimization/h2_test/coeff/h2_test_coeff_1.npy')
print(alpha_coeff)

[[ 0.63704121  0.80690766]
 [ 0.63704121 -0.80690766]]


In [11]:
onerdm_pyscf = driver.get_1pdm(ket)
twordm_pyscf = driver.get_2pdm(ket)


twordm_m = twordm_pyscf.reshape(4,4)

h1_int = h1e
g2_int = g2e

print(h1_int)
#print(g2_int)

one_body_en = np.einsum('ij,ij->', onerdm_pyscf, h1_int)

one_body_en_orbopt = 1.612888 - 3.728858
#print(onerdm_pyscf)
print(one_body_en)
print(one_body_en_orbopt)


#np.savetxt('/workspaces/MRA-OrbitalOptimization/h2_test/rdms/h2_test_1rdm.txt', onerdm_pyscf, fmt='%0.8f', delimiter=' ', header='2')
#np.savetxt('/workspaces/MRA-OrbitalOptimization/h2_test/rdms/h2_test_2rdm.txt', twordm_m, fmt='%0.8f', delimiter=' ', header='2')

[[-8.86175276e-01 -2.22044605e-16]
 [-1.11022302e-16 -6.68594018e-01]]
-1.7082146557839233
-2.11597


In [6]:
onerdm_sz = driver_sz.get_1pdm(ket_sz)
twordm_sz = driver_sz.get_2pdm(ket_sz)

alpha_1rdm = onerdm_sz[0]
beta_1rdm = onerdm_sz[1]

aa_2rdm = twordm_sz[0]
ab_2rdm = twordm_sz[1]
bb_2rdm = twordm_sz[2]

#print("Alpha alpha 2 rdm: ", aa_2rdm)
#print("Alpha Beta 2 rdm: ", ab_2rdm)
#print("Beta Beta 2 rdm: ", bb_2rdm)

np.save('/workspaces/MRA-OrbitalOptimization/h2_test/rdms/alpha_onerdm.npy', alpha_1rdm)
np.save('/workspaces/MRA-OrbitalOptimization/h2_test/rdms/beta_onerdm.npy', beta_1rdm)
np.save('/workspaces/MRA-OrbitalOptimization/h2_test/rdms/alpha_alpha_twordm.npy', aa_2rdm)
np.save('/workspaces/MRA-OrbitalOptimization/h2_test/rdms/alpha_beta_twordm.npy', ab_2rdm)
np.save('/workspaces/MRA-OrbitalOptimization/h2_test/rdms/beta_beta_twordm.npy', bb_2rdm)





In [2]:
a1_int = np.load('/workspaces/MRA-OrbitalOptimization/h2_test/output/alpha_htensor.npy')
b1_int = np.load('/workspaces/MRA-OrbitalOptimization/h2_test/output/beta_htensor.npy')
aa2_int = np.load('/workspaces/MRA-OrbitalOptimization/h2_test/output/alpha_alpha_gtensor.npy')
ab2_int = np.load('/workspaces/MRA-OrbitalOptimization/h2_test/output/alpha_beta_gtensor.npy')
bb2_int = np.load('/workspaces/MRA-OrbitalOptimization/h2_test/output/beta_beta_gtensor.npy')

In [ ]:
#do rhf - dmrg in Sz mode
H2 = gto.M(atom=[["H", 0, 0, 0],
                  ["H", 1.57, 0, 0]], basis = 'sto-3g', spin = 0)
rhf=scf.RHF(H2).run()

#norbs, nelecas = (2, 2)
#ref_ucas = mcscf.CASSCF(rhf, norbs, nelecas).mc1step()[0]
h1_int = (a1_int, b1_int)
g2_int = (aa2_int, ab2_int, bb2_int)

bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(rhf,
    ncore=0, ncas=None, g2e_symm=1)

driver_sz = DMRGDriver(scratch="./tmp3", symm_type=SymmetryTypes.SZ, n_threads=4)
driver_sz.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo_sz = driver_sz.get_qc_mpo(h1e=h1_int, g2e=g2_int, ecore=ecore, iprint=1)
ket_sz = driver_sz.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy_sz = driver_sz.dmrg(mpo_sz, ket_sz, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy SZ mode = %20.15f' % energy_sz)